In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Пакеты

Пакеты позволяют структурировать код на Python

Любая диретория имеющая внутри себя файл \_\_init\_\_.py является пакетом.

Весь синтаксис импорта сохраняется, но добавляется дотнотацию для модулей в пакете

Относительный импорт работате только для пакета, в котором вызывается импорт и с ним можно заменить имя пакета на одну точку . Но это все не работает в интерактивной оболочке Python. Т.е. это разумно использовать для модулей, использующих другие модули в пакете.

В модуле \_\_init\_\_.py имеет смысл инициализировать нечто глобальное (общие переменные, функции, кслассы) или определить фасад. Т.е. некую обеортку, содержащую информацию о структуре и устройстве пакета

In [1]:
# useful/bar/__init__.py
from .boo import *

__all__ = boo.__all__

# useful/__init__.py
from .foo import *
from .bar import *

__all__ = foo.__all__ + bar.__all__

SystemError: Parent module '' not loaded, cannot perform relative import

Плюсы и минусы использования фасадов в \_\_init\_\_.py

Плюсы
1. Пользователю не нужно запоминать внутреннюю структуру пакета и думать, с чем он рабоатет: модулем или пакетом
    - from urllib import urlopen или
    - from urllib.request import urlopen или
    - from urllib.requests import urlopen?
2. Интерфейс не зависит от деталей реализации - можно перемещать код между внутренними модулями и пакетами.
3. Одним словом, инкапсуляция и инженерное счастье.

Минусы
1. Накладные расходы на проход вглубину пакета. Особо тяжеловесные пакеты могут импортироваться порядка одной секунды.


Модуль можно выполнить честно как модуль добавив при вызове интерпритатора опцию *-m*

Дабы вызвать пакет, необходимо что бы в диретории пакета находился \_\_main\_\_.py

## Система импорта

В действительности импорт модулей и их методов осуществляется встроенной функцией \_\_import\_\_.

Если написать некую оборточную функцию, позволяющую получить информацию о текущем состоянии функции, можно в прямую посмотреть, что происходит в момент импорта того или иного метода:

In [2]:
def import_wrapper(name, *args, imp=__import__):
    print('importing ', name)
    return imp(name, *args)

In [3]:
import builtins

In [5]:
builtins.__import__ = import_wrapper

In [7]:
import collections

importing  session
importing  collections


На практике для импорта модуля по имени следует использовать функцию import_module из пакета importlib.

### Внутри функции \_\_import\_\_

In [ ]:
# Сначала для модуля создается пустой объект
import types
mod = types.ModuleType("useful")
# Затем байткод модуля вычисляется в пространстве имен созданного объекта
# (т.е. считывается, компилируется в байт-код и затем исполняется)
with open("./useful.py") as handle:
    source = handle.read()
    
code = compile(source, "useful.py", mode="exec")
exec(code, mod.__dict__)

# В завершение объект присваивается переменной с соответствующим именем
usefull = mod
useful # ~ import useful

При первом импорте исходный код модуля компилируется в байткод, который кэшируется в файле с расширением pyc.

In [2]:
def read_int(handle):
    return int.from_bytes(handle.read(4), 'little')

In [3]:
import collections

In [10]:
collections.__cached__

'c:\\users\\professional\\appdata\\local\\programs\\python\\python35\\lib\\collections\\__pycache__\\__init__.cpython-35.pyc'

In [11]:
handle = open(collections.__cached__, 'rb')
magic = read_int(handle) # тут содержиться информация для какой именно версии скомпилирован модуль
magic

168627478

In [12]:
mtime = read_int(handle)
print(mtime) # тут содержится дата последнего изменения

1465705922


In [8]:
import time
print(time.asctime(time.localtime(mtime)))

Sun Jun 12 07:32:02 2016


Все вышеперечисленное является первым этапом кэширования, ускоряющим импорт модулей.

Второй этап кэширования заключен в sys.modules

In [13]:
import sys

In [14]:
"collections" in sys.modules

True

Каждый модуль имеет некий идентификационный номер, позволяющий интерпритатору не производить перезагрузку модулей повторно (даже в том случае, если этого требует пользователь)

In [15]:
id(sys.modules['collections'])

2872834018664

In [16]:
import collections

In [17]:
id(sys.modules['collections'])

2872834018664

Модули кэшируются и лежат в sys.modules

!!!Проблема!!! Циклический импорт.  

Случается тогда, когда происходить импортирование двух зависимых друг от друга модулей.

Python не отслеживает отдельно циклический импотр ив случае него выводит ошибку ImportError, не более

1. Решением данной проблемы является вынесение общей функциональности модулей в отдельный модуль.
2. Осуществление локального импорта (внутри функции там, где это требуется)
3. Изменения модуля \_\_init\_\_.py так, что бы импорт проходил в конце модуля.

sys.path - список путей к модулям, пакетам или zip-архивам(python по умолчанию может импортировать из архива). Поиск в sys.path  осуществляется с лева на право. Таким образом не в коем случае нельзя называть собственные модули также, как уже имеющиеся в системе модули.

По уморлчанию в sys.path  уже что то есть, потому что инача ничего не будет работать. Помимо этого там лежить путь к site-packages(установленное через pip) и также переменные среды окружения PYTHONPATH

Есть нечто круче чем sys.path и это sys.meta_path

In [18]:
sys.meta_path

[_frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.WindowsRegistryFinder,
 _frozen_importlib_external.PathFinder,

Это искатели и загрузчики модулей

Finder должен реализовывать метод find_spec, принимающий имя модуля и возвращающий ModuleSpec или None, если модуль не был найден.

In [19]:
builtit_finder, _, _, path_finder, _ = sys.meta_path

In [20]:
builtit_finder.find_spec("itertools")

ModuleSpec(name='itertools', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in')

In [21]:
builtit_finder.find_spec('enum') # Ничего не вернул, потому что модуль не встроенный

In [22]:
path_finder.find_spec("enum") # Нашел, потому что модуль не встроенный

ModuleSpec(name='enum', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000029CE6E52208>, origin='c:\\users\\professional\\appdata\\local\\programs\\python\\python35\\lib\\enum.py')

In [24]:
builtit_finder.find_spec("math")

ModuleSpec(name='math', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in')

ModuleSpec содержит все необходимую информацию для загрузки модуля

In [25]:
spec = path_finder.find_module("collections")

In [26]:
spec.name

'collections'

In [27]:
spec.path

'c:\\users\\professional\\appdata\\local\\programs\\python\\python35\\lib\\collections\\__init__.py'

In [28]:
spec.source_to_code

<bound method SourceLoader.source_to_code of <_frozen_importlib_external.SourceFileLoader object at 0x0000029CE6E43CC0>>

In [29]:
spec.load_module

<bound method FileLoader.load_module of <_frozen_importlib_external.SourceFileLoader object at 0x0000029CE6E43CC0>>

Необходимость подобного знания можно описать на коротком примере:

Имеется две реализации одного и тогоже модуля. Одна быстрая (реализованная на каком либо ЯП) и одна на Python.

Для вылавливания возможной ошибки импорта можно реализовать следующий код:

In [ ]:
try:
    import _useful_speedups as useful
except ImportError:
    import useful

Некорректность заключается в том, что ошибка ImportError может быть результатом совсем не проблем с нативным модулем, а по причине нашей собственной криворукости.

Более надежным вариантом будет следующий код:

In [ ]:
from importlib.util import find_spec

if find_spec("_useful_speedups"):
    import _useful_speedups as useful
else:
    import useful

Функция find_spec обходит sys.meta_path и последовательно вызывает одноименный метод у каждого из импортеров

Loader должен реализовывать два метода:
+ create_module - для создания пустого модуля
+ exec_module - для заполнения пустого модуля

In [30]:
from importlib.util import find_spec

In [31]:
spec = find_spec("enum")

In [32]:
mod = spec.loader.create_module(spec)

In [34]:
print(mod) # Пустой модуль

None


In [35]:
from importlib.util import module_from_spec

In [36]:
mod = module_from_spec(spec)

In [37]:
mod

<module 'enum' from 'c:\\users\\professional\\appdata\\local\\programs\\python\\python35\\lib\\enum.py'>

In [38]:
dir(mod)

['__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [39]:
spec.loader.exec_module(mod)

In [40]:
dir(mod)

['DynamicClassAttribute',
 'Enum',
 'EnumMeta',
 'IntEnum',
 'MappingProxyType',
 'OrderedDict',
 '_EnumDict',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_is_descriptor',
 '_is_dunder',
 '_is_sunder',
 '_make_class_unpicklable',
 '_reduce_ex_by_name',
 'sys',
 'unique']

#### Пример: Автоматическая установка

In [41]:
import subprocess
import sys
from importlib.util import find_spec
from importlib.abc import MetaPathFinder

In [42]:
"""
Искатель будет добавлен в meta_path так как наследуется от MetaPathFinder.
Если модуль не находится, то запускается команда pip и устанавливает модуль.

"""
class AutoInstall(MetaPathFinder):
    _loaded = set()
    
    @classmethod
    def find_spec(cls, name, path=None, target=None):
        if path is None and name not in cls._loaded:
            print('Installing ', name)
            cls._loaded.add(name)
            try:
                subprocess.check_output([
                    sys.executable, "-m", "pip", "install", "name"
                ])
                return find_spec(name)
            except Exception:
                print("Failed")
        return None

Помимо указанных списков, имеется еще ли sys.path_hooks. В ней лежал определенные функции. Одна из которых умеет читать(импортировать) zip-архивы, а вторая умеет импортировать из всего остального

In [43]:
sys.path_hooks

[zipimport.zipimporter,
 <function _frozen_importlib_external.FileFinder.path_hook.<locals>.path_hook_for_FileFinder(path)>]

#### Пример: Удаленный импорт

In [47]:
"""
Подаваемая строка будет проверятся на соответствие URL(начинается HTTP и HTTPs)
Если все соответствует, сслыка будет открыта urlopen.
После этого, на странице будет найдены все перечисленные модули python
"""
import re
import sys
from urllib.request import urlopen

def url_hook(url):
    if not url.startwith(('http','https')):
        raise ImportError
    with urlopen(url) as page:
        data = page.read().decode("utf-8")
    filenames = re.findall("[a-zA-Z_][a-zA-Z0-0_]*.py", data)
    modnames = {name[:-3] for name in filenames}
    return URLFinder(url, modnames)

sys.path_hooks.append(url_hook)

from importlib.abc import PathEntryFinder
from importlib.util import spec_from_loader

class URLFinder(PathEntryFinder):
    def __init__(self, url, available):
        self.url = url
        self.available = available
        
    def find_spec(self, name, target=None):
        if name in self.available:
            origin = '{}/{}.py'.format(self.url, name)
            loader = URLLoader()
            return spec_from_loader(name, loader, origin=origin)
        else:
            return None
        
        
class URLLoader:
    def create_module(self, target):
        return None
    
    def exec_module(self, module):
        with urlopen(module.__spec__.origin) as page:
            source = page.read()
        code = compile(source, module.__spec__.origin, mode='exec')
        exec(code, module.__dict__)